In [53]:
import tweepy
import csv
import pandas as pd
import json

from nltk.corpus import stopwords, reuters
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt
import datetime
from ibm_watson import ToneAnalyzerV3
from pandas import json_normalize
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from dotenv import load_dotenv

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

all_tweets = []

In [54]:
def load_vaccine_csv():
    df = pd.read_csv('./covidvaccine.csv')
    df = df[['user_name','user_location','user_followers',	'user_friends',	'user_favourites',	'user_verified',	'date',	'text',	'source']]
    
    # '18-08-2020 12:55'
    # df.index = pd.to_datetime(df['date'])
    # for managebility we remove the hashtag, but it might have a future use-case
    # is_retweet is all False in this dataset
    # df = df[df['is_retweet'] == False]
    # df = df.dropna()
    return df

In [55]:
def try_parse(dateString):
    # return dateString
    # print(dateString)
    try:
        return datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M:%S')
    except Exception as e:
        try:
            return datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M')
        except Exception as e:
            try:
                return datetime.datetime.strptime(dateString, '%d-%m-%Y %H:%M')
            except Exception as e:
                return datetime.datetime.strptime(dateString, '%d-%m-%Y %H:%M:%S')


In [56]:
# 354390
# 354390
# 216910
df = load_vaccine_csv()
# df.index = pd.to_datetime(df.date, errors='coerce')

#18-08-2020 12:55
df = df[df.date.notnull()]
df = df[df.date.str.contains("-")]

df['date'] = df['date'].apply(lambda t: try_parse(t))
df['xxx'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M')
# df = df[df.date.isnull()]

df = df.sort_index()
df

,user_name,user_location,user_followers,user_friends,user_favourites,user_verified,date,text,source,xxx
0,MyNewsNE,Assam,64.0,11,110,False,2020-08-18 12:55:00,Australia to Manufacture Covid-19 Vaccine and ...,Twitter Web App,2020-08-18 12:55:00
1,Shubham Gupta,NaN,1.0,17,0,False,2020-08-18 12:55:00,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,Twitter for Android,2020-08-18 12:55:00
2,Journal of Infectiology,NaN,143.0,566,8,False,2020-08-18 12:46:00,Deaths due to COVID-19 in Affected Countries\n...,Twitter Web App,2020-08-18 12:46:00
3,Zane,NaN,29.0,25,620,False,2020-08-18 12:45:00,@Team_Subhashree @subhashreesotwe @iamrajchoco...,Twitter for Android,2020-08-18 12:45:00
4,Ann-Maree O’Connor,"Adelaide, South Australia",83.0,497,10737,False,2020-08-18 12:45:00,@michellegrattan @ConversationEDU This is what...,Twitter Web App,2020-08-18 12:45:00
...,...,...,...,...,...,...,...,...,...,...
354385,pinakbet808,"San Francisco, CA",0.0,83,0,False,2021-02-15 00:01:02,I called my lola to wish her a Happy Valentine...,Twitter for iPhone,2021-02-15 00:01:02
354386,UCAN Indonesia,Jakarta,2954.0,1023,2,False,2021-02-15 00:00:56,Cambodia’s Ministry of Health approves emergen...,SocialPilot.co,2021-02-15 00:00:56
354387,UCAN India,India,1771.0,270,65,False,2021-02-15 00:00:56,Cambodia’s Ministry of Health approves emergen...,SocialPilot.co,2021-02-15 00:00:56
354388,VaccinatePA,Pennsylvania,1191.0,44,44,False,2021-02-15 00:00:09,"Thanks for helping us get the word out, @abc27...",Twitter Web App,2021-02-15 00:00:09


In [57]:
df2 = df.dropna()
df2

,user_name,user_location,user_followers,user_friends,user_favourites,user_verified,date,text,source,xxx
0,MyNewsNE,Assam,64.0,11,110,False,2020-08-18 12:55:00,Australia to Manufacture Covid-19 Vaccine and ...,Twitter Web App,2020-08-18 12:55:00
4,Ann-Maree O’Connor,"Adelaide, South Australia",83.0,497,10737,False,2020-08-18 12:45:00,@michellegrattan @ConversationEDU This is what...,Twitter Web App,2020-08-18 12:45:00
6,Rajesh Tadepalli,"Hyderabad, India",918.0,2561,31837,False,2020-08-18 12:34:00,@PrivilRodrigues @yatish57 @deepkaranahuja @sh...,Twitter for Android,2020-08-18 12:34:00
7,AKisASocialisolationist wash yer damn hands,The Great Pacific Northwest,2321.0,3236,264351,False,2020-08-18 12:30:00,"@MSNBC Well, let’s qualify that: would anyone ...",Twitter for iPhone,2020-08-18 12:30:00
8,Dr. Joseph Santoro,"Washington, DC 20009",19091.0,20986,128119,False,2020-08-18 12:15:00,"Most countries, without the ability to make #V...",Hootsuite Inc.,2020-08-18 12:15:00
...,...,...,...,...,...,...,...,...,...,...
354384,Dr Mahendra G Patel We-are-ALL-AT-RISK #COVID19,"Yorkshire and The Humber, England",5245.0,1296,25625,False,2021-02-15 00:03:11,After weeks away now dipping in lightly &amp; ...,Twitter for iPhone,2021-02-15 00:03:11
354385,pinakbet808,"San Francisco, CA",0.0,83,0,False,2021-02-15 00:01:02,I called my lola to wish her a Happy Valentine...,Twitter for iPhone,2021-02-15 00:01:02
354386,UCAN Indonesia,Jakarta,2954.0,1023,2,False,2021-02-15 00:00:56,Cambodia’s Ministry of Health approves emergen...,SocialPilot.co,2021-02-15 00:00:56
354387,UCAN India,India,1771.0,270,65,False,2021-02-15 00:00:56,Cambodia’s Ministry of Health approves emergen...,SocialPilot.co,2021-02-15 00:00:56


In [58]:
load_vaccine_csv().dtypes


user_name           object
user_location       object
user_followers     float64
user_friends        object
user_favourites     object
user_verified       object
date                object
text                object
source              object
dtype: object

In [59]:

df.describe()
# ####input your credentials here
# consumer_key = 'SCOPBRKeG4nRCEa7XkMoqQ'
# consumer_secret = 'RYInMkLiNyg0iKC3g89Y0f8g8kbNFSsNjZXNYBYILQ'
# access_token = '107275871-PtAa9t7OJX82IEbGjdOzaFMgeCWQyU8haNPhN4mD'
# access_token_secret = 'Q691rTrztjXeYAP5FEA5kZWRhuFRtH8sq6msPgUeD1TzI'

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth,wait_on_rate_limit=True)
# #####United Airlines
# # Open/Create a file to append data
# csvFile = open('ua.csv', 'a')
# #Use csv Writer
# csvWriter = csv.writer(csvFile)

# all_tweets = []

# for tweet in tweepy.Cursor(api.search,q="#dulux",count=100,
#                            lang="en",
#                            since="2015-11-03").items():
#     print (tweet.created_at, tweet.text)
#     all_tweets.append(tweet.text)
#     csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

,user_followers
count,1.374770e+05
mean,5.157729e+04
std,6.479442e+05
min,0.000000e+00
25%,1.390000e+02
50%,6.030000e+02
75%,2.664000e+03
max,1.474791e+07


In [60]:
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nimendrapethiyagoda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [61]:
def process_text(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', doc)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return ' '.join(output)

In [62]:
def analyse_sentiment(df):
    load_dotenv()
    tone_api = os.getenv("TONE_ANALYZER_APIKEY")
    tone_url = os.getenv("TONE_ANALYZER_URL")
    authenticator = IAMAuthenticator(tone_api)
    tone_analyzer = ToneAnalyzerV3(
        version="2017-09-21",
        authenticator=authenticator
    )

    utterances = [{"text": u, "user": "guest"} for u in df['text'].to_numpy()]

    # Analyze utterances using the 'tone_chat()' method
    utterance_analysis = tone_analyzer.tone_chat(
        utterances=utterances, content_language="en", accept_language="en"
    ).get_result()
    # print(json.dumps(utterance_analysis, indent=2))
    chat_tone_df = json_normalize(
        data=utterance_analysis["utterances_tone"],
        record_path=["tones"],
        meta=["utterance_id", "utterance_text"],
    )
    return chat_tone_df

In [82]:
def analyse_sentimemnt_vader(df):
    
    sentiments = []

    for index, row in df.iterrows():
        try:
            text = row["text"]
            # date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
            
            sentiments.append({
                "text": text,
                "date": row['date'],
                "user_location":row['user_location'],
                "user_followers":row['user_followers'],
                "user_friends":row['user_friends'],
                "user_favourites":row['user_favourites'],
                "user_name": row['user_name'],
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu
                
            })
            
        except AttributeError:
            pass
        
    # Create DataFrame
    ret_df = pd.DataFrame(sentiments)

    # Reorder DataFrame columns
    cols = ["date","user_name", "user_location","user_followers", "user_friends", "user_favourites", "text", "compound", "positive", "negative", "neutral"]
    ret_df = ret_df[cols]
    ret_df.set_index('date', inplace=True)
    ret_df = ret_df.sort_index()

    return ret_df

In [84]:
# df['text'].to_numpy()
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
chat_tone_df = analyse_sentimemnt_vader(df2.head(500))
chat_tone_df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nimendrapethiyagoda/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,user_name,user_location,user_followers,user_friends,user_favourites,text,compound,positive,negative,neutral
date,,,,,,,,,,
2020-08-13 18:07:00,Thecortexmedia.Com,"Jammu, Jammu And Kashmir",5.0,3.0,70.0,Russia Beat America And Launched first #Covid...,0.1280,0.077,0.000,0.923
2020-08-13 18:16:00,Frank Hill,"Raleigh, North Carolina",1223.0,1845.0,2495.0,"Odd, isn't it, that just 5 months ago, many po...",-0.3182,0.000,0.113,0.887
2020-08-13 18:17:00,CHARMARKE ABDOULKADER HADJI 查马克,Djibouti,74.0,223.0,649.0,"#CovidVaccine\nPutin :All Eyes on You ,please ...",0.3182,0.150,0.000,0.850
2020-08-13 18:24:00,Andrea Pruijssers,"Nashville, TN, USA",158.0,85.0,443.0,I enrolled in the Phase III #Moderna vaccine t...,0.0000,0.000,0.000,1.000
2020-08-13 18:30:00,Thecortexmedia.Com,"Jammu, Jammu And Kashmir",5.0,3.0,70.0,See The Full Price of Russia Vaccine In Your C...,0.0000,0.000,0.000,1.000
...,...,...,...,...,...,...,...,...,...,...
2020-08-18 12:15:00,Dr. Joseph Santoro,"Washington, DC 20009",19091.0,20986.0,128119.0,"Most countries, without the ability to make #V...",-0.3532,0.098,0.195,0.707
2020-08-18 12:30:00,AKisASocialisolationist wash yer damn hands,The Great Pacific Northwest,2321.0,3236.0,264351.0,"@MSNBC Well, let’s qualify that: would anyone ...",0.5859,0.202,0.000,0.798
2020-08-18 12:34:00,Rajesh Tadepalli,"Hyderabad, India",918.0,2561.0,31837.0,@PrivilRodrigues @yatish57 @deepkaranahuja @sh...,0.0000,0.000,0.000,1.000


In [77]:
from geopy.geocoders import Nominatim
app = Nominatim(user_agent="JournalDev")
location = app.geocode("Austin, Texas")
location

Location(Austin, Travis County, Texas, United States, (30.2711286, -97.7436995, 0.0))